In [ ]:
%reload_ext autoreload

%autoreload 2

In [ ]:
import os
import directlfq
import alphabase.test_data_downloader

# Get the directory of the directlfq package
directlfq_dir = os.path.dirname(directlfq.__file__)

# Navigate up two levels to reach the project root
project_root = os.path.abspath(os.path.join(directlfq_dir, '..'))

output_dir = os.path.join(project_root, 'test_data', 'system_tests')

os.makedirs(output_dir, exist_ok=True)
# Now use the absolute path in the DataShareDownloader call
alphabase.test_data_downloader.DataShareDownloader(url="https://datashare.biochem.mpg.de/s/zjSHOAzKBaF2Z73", output_dir=output_dir).download()


In [ ]:
INPUT_FILE = f"{output_dir}/diaPASEF_diann/report.tsv.top3added.tsv.zip"

DIRECTLFQ_FILE_PRECURSORS_MS1_MS2 = f"{INPUT_FILE}.diann_precursor_ms1_and_ms2.protein_intensities.tsv"
DIRECTLFQ_FILE_PRECURSORS = f"{INPUT_FILE}.diann_precursors.protein_intensities.tsv"
DIRECTLFQ_FILE_FRAGIONS = f"{INPUT_FILE}.diann_precursor_ms1_and_ms2.protein_intensities.tsv"


SAMPLEMAP  = f"{output_dir}/diaPASEF_diann/samplemap.tsv"

In [ ]:
import directlfq.lfq_manager as lfqmgr
input_types_to_use  = ["diann_fragion_isotopes_topn", "diann_precursor_ms1_and_ms2", "diann_precursors", "diann_fragion_isotopes"]
for input_type in input_types_to_use:
	lfqmgr.run_lfq(input_file=INPUT_FILE, input_type_to_use=input_type)

In [ ]:
import directlfq.benchmarking as lfqbenchmark
import directlfq.utils as lfqutils


samples_used = lfqutils.get_samples_used_from_samplemap_file(SAMPLEMAP, cond1="45ng", cond2="15ng")


restable_directlfq_ms1_and_ms2 = lfqbenchmark.ResultsTableDirectLFQ(input_file=DIRECTLFQ_FILE_PRECURSORS_MS1_MS2, input_name="directLFQ", samples_c1=samples_used[0], samples_c2=samples_used[1])
restable_directlfq_precursors = lfqbenchmark.ResultsTableDirectLFQ(input_file=DIRECTLFQ_FILE_PRECURSORS, input_name="directLFQ", samples_c1=samples_used[0], samples_c2=samples_used[1])
restable_directlfq_fragions = lfqbenchmark.ResultsTableDirectLFQ(input_file=DIRECTLFQ_FILE_FRAGIONS, input_name="directLFQ", samples_c1=samples_used[0], samples_c2=samples_used[1])


restables = [restable_directlfq_ms1_and_ms2, restable_directlfq_precursors, restable_directlfq_fragions]
restable_names = ["directLFQ (precursors, MS1+MS2)", "directLFQ (precursors)", "directLFQ (fragions)"]

organism_annotator_diann = lfqbenchmark.OrganismAnnotatorDIANN(mapping_file=INPUT_FILE)
for restable in restables:
    organism_annotator_diann.annotate_table_with_organism(restable)
    

methodname2formatted_df = {'directLFQ_precursor_ms1_and_ms2' : restable_directlfq_ms1_and_ms2.formated_dataframe, 'directLFQ_precursor' : restable_directlfq_precursors.formated_dataframe,
                        'directLFQ_fragions' : restable_directlfq_fragions.formated_dataframe}

In [ ]:
import numpy as np
import directlfq.test_utils as testutils

organism2expectedfc={"YEAST" : np.log2(3), "HUMAN" : 0}
organism2CI95={"YEAST" : 0.93, "HUMAN" : 0.3}

for method, formatted_df in methodname2formatted_df.items():
	testutils.RatioChecker(formatted_df=formatted_df, organism2expectedfc=organism2expectedfc, organism2CI95=organism2CI95)
	print(f"Method {method} passed the test")
	



In [ ]:
import seaborn as sns
import directlfq.visualizations
import directlfq.benchmarking
import matplotlib.pyplot as plt
import numpy as np


class MultiOrganismMultiMethodBoxPlotAdapted(directlfq.visualizations.MultiOrganismMultiMethodBoxPlot):
    def plot_boxplot(self):
        color_palette = sns.color_palette(self._colorlist_hex, n_colors=len(self._fcs_to_expect))
        sns.boxplot(data=self._method_ratio_results_table, x="method", y = "log2fc", hue= "organism", palette=color_palette, hue_order=self._organisms_to_plot, ax=self.ax, linewidth=0.8)




num_results = len(restables)

fig, axes = plt.subplots(1, num_results+1,figsize=(4*num_results, 4.363))

organisms_to_plot = [ "YEAST", "HUMAN"]
fcs_to_expect = [  np.log2(3), 0]

merged_table = directlfq.benchmarking.ResultsTableMerger(method_name2results_df=methodname2formatted_df).merged_table


MultiOrganismMultiMethodBoxPlotAdapted(method_ratio_results_table=merged_table, ax = axes[0], organisms_to_plot=organisms_to_plot,fcs_to_expect= fcs_to_expect)


for idx in range(num_results):
    restable = restables[idx]
    name = restable_names[idx]
    ax = axes[idx+1]
    directlfq.visualizations.MultiOrganismIntensityFCPlotter(ax=ax, resultstable_w_ratios=restable, organisms_to_plot = organisms_to_plot, fcs_to_expect= fcs_to_expect, title=name)

